In [ ]:
!pip install -q langchain
!pip install -qU langchain-groq
!pip install -qU langchain-text-splitters
!pip install --upgrade --quiet  fastembed
!pip install -q langchain_community
!pip install -q chromadb
!pip install -q langchain-chroma

In [ ]:

import getpass
import os

from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = ""

model = ChatGroq(model="llama3-8b-8192")

# Text splitter

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load example document
with open("transcribe.txt") as f:
    state_of_the_union = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=300,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([state_of_the_union])

for t in texts:
    print(t)


In [ ]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain import LLMChain
from langchain_chroma import Chroma

embed_model = FastEmbedEmbeddings()

vs = Chroma.from_documents(
    documents=texts,
    embedding=embed_model,
    persist_directory="chroma_db_dir",  # Local mode with in-memory storage only
    collection_name="stanford_report_data"
)


In [ ]:
vectorstore = Chroma(embedding_function=embed_model,
                     persist_directory="chroma_db_dir",
                     collection_name="stanford_report_data")
retriever=vectorstore.as_retriever(search_kwargs={'k': 7})

In [ ]:
retriever.invoke("a que esta asociado un lead?")

In [ ]:
from langchain.prompts import PromptTemplate

custom_prompt_template = """Usa la siguiente información para responder a la pregunta del usuario.
Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.

Contexto: {context}
Pregunta: {question}

Solo devuelve la respuesta útil a continuación y nada más y responde siempre en español
Respuesta útil:
"""
prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context', 'question'])

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=model,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs={"prompt": prompt})

In [ ]:
response = qa.invoke({"query": "Hablame sobre el Lead?"})
response["result"]

In [ ]:
response = qa.invoke({"query": "Que es una unidad y que relacion tiene con el Lead?"})
response["result"]

In [ ]:
response = qa.invoke({"query": "que es una cotización?"})
response["result"]

In [ ]:


from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Crear un prompt template
prompt_template = PromptTemplate(input_variables=["question"], template="Pregunta: {question}\nRespuesta:")

# Crear una cadena LLM con el modelo Groq
chain = LLMChain(llm=model, prompt=prompt_template)

responses = []

for text in texts:
    response = chain.invoke("resume lo siguiente en 200 palabras: " + text.page_content)    
    responses.append(response)

# Hacer una pregunta

print(responses)


In [123]:
# Concatenar resúmenes de manera más eficiente
summaries = "\n".join([r["text"] for r in responses])

# Si realmente necesitas cortar el resumen a la mitad
summaries = summaries[:len(summaries) // 2]

# Construir el prompt
prompt_to_resume = f"""
Combina los resúmenes de las diferentes secciones del documento y elabora un resumen general conciso. Asegúrate de capturar los 
puntos clave de cada sección, eliminando redundancias y preservando la coherencia y el flujo de la información.

{summaries}
"""

# Invocar el modelo con el prompt
response = chain.invoke(prompt_to_resume)


In [125]:
response

{'question': '\nCombina los resúmenes de las diferentes secciones del documento y elabora un resumen general conciso. Asegúrate de capturar los \npuntos clave de cada sección, eliminando redundancias y preservando la coherencia y el flujo de la información.\n\nLa transcripción es el proceso de convertir un contenido audio o video en texto. En este ejemplo, se transcribe una conversación entre alguien y Roger. La transcripción comienza con un saludo y una breve charla informal, donde se pregunta sobre el estado del otro y se comenta que la persona se siente un poco mejor después de hablar sobre algo. La transcripción incluye palabras y frases como "¿Qué tal?", "Estoy avanzando", "Aprovechando", "Ya me siento un poco mejor" y "Lo que hablamos ayer". La transcripción es un proceso importante en various campos, como la investigación, la educación y la comunicación, ya que permite analizar y comprender mejor el contenido audio o video.\nLa respuesta es difícil de entender debido a la charla